# Examples of utils provided by ReGraph's Neo4jHierarchy module

In [1]:
import networkx as nx
from regraph import Rule, plot_rule


from regraph.neo4j.hierarchy import Neo4jHierarchy
import regraph.neo4j.cypher_utils as cypher

## Initializing Neo4j database

0. When installing neo4j you will be asked to choose login/password for you dbs (here its "neo4j"/"admin"), if you choose other -- change in the cell below.
1. To start neo4j server run `sudo service neo4j start`
2. Check status by running `sudo service neo4j status`. Here you can check the _bolt_ port, change the cell below if different from 7687
3. You can query the db by using the neo4j browser, the address can be found also in the result of 'status', e.g. "Remote interface available at http://localhost:7474/".

In [2]:
# initialize the neo4j driver, wrappped into Neo4jHierarchy object
h = Neo4jHierarchy(uri="bolt://localhost:7687", user="neo4j", password="admin")
h._clear()

## Hierarchy

A hierarchy is given by graphs connected by typing (homorphism). 

Each graph in the database is represented by a unique label which enables us to match quickly the nodes of this graph. Each node of a graph has then the label *graphId*. We can match these nodes by using the query:

> MATCH (n:graphLabel) RETURN n


The edges of a graph are labeled as *edge* and the typing edges are labeled as *typing*. We can then easily find the image of a node with the query:

> OPTIONAL MATCH (n:graphLabel)-[:typing]->(m) RETURN m

The hierarchy skeleton is represented by nodes labeled as hierarchyNodes. It enables us to know which graphs are the ancestors of a graph we want to rewrite for example.

## Adding a graph to the hierarchy (Action Graph)

Here we create a first graph called *ActionGraph*. This graph represent people and the relationship between them.

In [3]:
nodes = [
    ("a", {"name": {"Jack"}, "age": {43}, "hobby": {"hiking", "music"}}), 
    ("b", {"name": {"Robert"}, "age": {35}, "hobby": {"sport", "music"}}),
    ("c", {"name":{ "Paul"}, "age": {18}}),
    ("d", {"name": {"Joe"}, "age": {12}}),
    ("e", {"name": {"Paul"}, "age": {18}})
]
edges = [
    ("a", "b", {"type": {"friends"}}), 
    ("a", "c", {"type": {"colleagues"}}),
    ("b", "c", {"type": {"colleagues"}}),
    ("d", "a", {"type": {"child_of"}}), 
    ("d", "e", {"type": {"enemies"}}),
    ("e", "e", {"type": {"likes"}})
]
h.add_graph('actionGraph', nodes, edges)

CREATE CONSTRAINT ON (n:actionGraph) ASSERT n.id IS UNIQUE
OPTIONAL MATCH (same_id_node:actionGraph) 
WHERE same_id_node.id = 'a' 
FOREACH(new_count 
	IN CASE WHEN same_id_node IS NOT NULL
	THEN [coalesce(same_id_node.count, 0) + 1]
	ELSE [] END | 
		SET same_id_node.count = new_count) 
WITH same_id_node 
UNWIND
	CASE WHEN same_id_node IS NOT NULL
	THEN ['a' + same_id_node.count]
	ELSE ['a'] END AS new_id_a 
		CREATE (a:actionGraph { id : new_id_a }) 
SET a.name=['Jack']
SET a.age=[43]
SET a.hobby=['hiking', 'music']
WITH a, new_id_a OPTIONAL MATCH (same_id_node:actionGraph) 
WHERE same_id_node.id = 'b' 
FOREACH(new_count 
	IN CASE WHEN same_id_node IS NOT NULL
	THEN [coalesce(same_id_node.count, 0) + 1]
	ELSE [] END | 
		SET same_id_node.count = new_count) 
WITH same_id_node 
UNWIND
	CASE WHEN same_id_node IS NOT NULL
	THEN ['b' + same_id_node.count]
	ELSE ['b'] END AS new_id_b 
		CREATE (b:actionGraph { id : new_id_b }) 
SET b.name=['Robert']
SET b.age=[35]
SET b.hobby=['sport', 'mus

## Adding a second graph to the herarchy (Meta Model)

Here we create a second graph called *MetaModel*. Which represent the types of people (*Adult* or *Child*) and the relationships between them.

In [4]:
nodes = [
    ("a", {"type": {"Adult"}, "age": {43, 35, 18}, "name": {"Paul", "Jack", "Robert"}, "hobby": {"hiking", "music", "sport"}}),
    ("b", {"type": {"Child"}, "age": {12, 18}, "name": {"Paul", "Joe"}})
]
edges = [
    ("a", "a", {"type": {"friends", "colleagues"}}),
    ("b", "a", {"type": {"child_of"}}),
    ("b", "b", {"type": {"friends", "enemies"}})
]
h.add_graph('metaModel', nodes, edges)

CREATE CONSTRAINT ON (n:metaModel) ASSERT n.id IS UNIQUE
OPTIONAL MATCH (same_id_node:metaModel) 
WHERE same_id_node.id = 'a' 
FOREACH(new_count 
	IN CASE WHEN same_id_node IS NOT NULL
	THEN [coalesce(same_id_node.count, 0) + 1]
	ELSE [] END | 
		SET same_id_node.count = new_count) 
WITH same_id_node 
UNWIND
	CASE WHEN same_id_node IS NOT NULL
	THEN ['a' + same_id_node.count]
	ELSE ['a'] END AS new_id_a 
		CREATE (a:metaModel { id : new_id_a }) 
SET a.type=['Adult']
SET a.age=[43, 18, 35]
SET a.name=['Jack', 'Paul', 'Robert']
SET a.hobby=['hiking', 'sport', 'music']
WITH a, new_id_a OPTIONAL MATCH (same_id_node:metaModel) 
WHERE same_id_node.id = 'b' 
FOREACH(new_count 
	IN CASE WHEN same_id_node IS NOT NULL
	THEN [coalesce(same_id_node.count, 0) + 1]
	ELSE [] END | 
		SET same_id_node.count = new_count) 
WITH same_id_node 
UNWIND
	CASE WHEN same_id_node IS NOT NULL
	THEN ['b' + same_id_node.count]
	ELSE ['b'] END AS new_id_b 
		CREATE (b:metaModel { id : new_id_b }) 
SET b.type=['Chil

## Typing

Here we create a typing of the *ActionGraph* by the *MetaModel*.

In [5]:
mapping = {
    "a":"a",
    "b":"a",
    "c":"a",
    "d":"b",
    "e":"b"
}
h.add_typing('actionGraph', 'metaModel', mapping)

CREATE CONSTRAINT ON (n:actionGraph) ASSERT n.id IS UNIQUE
CREATE CONSTRAINT ON (n:metaModel) ASSERT n.id IS UNIQUE


## Classic update functions

We still can use the modification functions on these graphs.

In [6]:
#ag.merge_nodes1(["b", "c"])

In [7]:
#ag.clone_node("e")

## Graph rewriting and propagation up

In [8]:
pattern = nx.DiGraph()
pattern.add_nodes_from(["x", ("y",  {"type": {"Child"}, "age": {12, 18}, "name" : {"Paul", "Joe"}})])
pattern.add_edges_from([("y", "x",  {"type": {"child_of"}}), ("y", "y")])

In [9]:
rule = Rule.from_transform(pattern)
rule.inject_remove_node_attrs("y",  {"age": {18}})
rule.inject_remove_edge_attrs("y", "x",  {"type": {"child_of"}})
#rule.inject_remove_node("x")
#rule.inject_remove_edge("y","y")
#rule.inject_clone_node("y")

In [10]:
instances = h.find_matching('metaModel', pattern)
print("Instances: ", instances)

CREATE CONSTRAINT ON (n:metaModel) ASSERT n.id IS UNIQUE
MATCH (x:metaModel), (y:metaModel), (y)-[:edge]->(x), (y)-[:edge]->(y)
WHERE id(x) <> id(y)
 AND 'Child' IN y.type AND 18 IN y.age AND 12 IN y.age AND 'Paul' IN y.name AND 'Joe' IN y.name
RETURN x, y
Instances:  [{'x': 'a', 'y': 'b'}]


In [11]:
h.rewrite('metaModel', rule, instances[0])

CREATE CONSTRAINT ON (n:metaModel) ASSERT n.id IS UNIQUE
// Match nodes and edges of the instance 
MATCH (lhs_x:metaModel { id : 'a'}) , (lhs_y:metaModel { id : 'b'})  , (lhs_y)-[lhs_y_lhs_x:edge]->(lhs_x), (lhs_y)-[lhs_y_lhs_y:edge]->(lhs_y)


// Renaming vars to correspond to the vars of P
WITH lhs_y_lhs_y, lhs_y_lhs_x , lhs_x as p_x, lhs_y as p_y 

// Removing properties from node 'y' of P 
FOREACH(dummy IN CASE WHEN 'age' IN keys(p_y) THEN [1] ELSE [] END |
	SET p_y.age = filter(v in p_y.age WHERE NOT v IN ['18'])
	FOREACH(dumy2 IN CASE WHEN size(p_y.age)=0 THEN [1] ELSE [] END |
		REMOVE p_y.age))


// Removing properties from edge y->x of P 
WITH lhs_y_lhs_x, p_x, p_y, lhs_y_lhs_y MATCH (p_y)-[p_y_p_x:edge]->(p_x)
FOREACH(dummy IN CASE WHEN 'type' IN keys(p_y_p_x) THEN [1] ELSE [] END |
	SET p_y_p_x.type = filter(v in p_y_p_x.type WHERE NOT v IN ['child_of'])
	FOREACH(dumy2 IN CASE WHEN size(p_y_p_x.type)=0 THEN [1] ELSE [] END |
		REMOVE p_y_p_x.type))


// Renaming vars to corr

(<regraph.neo4j.hierarchy.Neo4jHierarchy at 0x7fc5fbdc17b8>,
 {'x': 'a', 'y': 'b'})

## Graph rewritting and propagation down

In [12]:
pattern = nx.DiGraph()
pattern.add_nodes_from([("w", {"name":{"Paul"}}), "x", "y", "z"])
pattern.add_edges_from([("x", "w"), ("y", "x"), ("y", "z"), ("z","z")])

In [13]:
rule = Rule.from_transform(pattern)
rule.inject_add_node_attrs("x", {"age":{10}})
rule.inject_add_edge_attrs("x", "w", {"relation":{"brothers"}})
rule.inject_add_node("new_node", {"name":{"Will"}})
rule.inject_add_edge("new_node", "y")
merged_node = rule.inject_merge_nodes(["w", "z"])

In [14]:
rhs_typing = {
    'metaModel': {'new_node':'a', merged_node: 'a'}
}

In [15]:
instances = h.find_matching('actionGraph', pattern)
print("Instances: ", instances)

CREATE CONSTRAINT ON (n:actionGraph) ASSERT n.id IS UNIQUE
MATCH (w:actionGraph), (x:actionGraph), (y:actionGraph), (z:actionGraph), (x)-[:edge]->(w), (y)-[:edge]->(x), (y)-[:edge]->(z), (z)-[:edge]->(z)
WHERE id(w) <> id(x) AND id(x) <> id(y) AND id(y) <> id(z)
 AND 'Paul' IN w.name
RETURN w, x, y, z
Instances:  [{'w': 'c', 'x': 'a', 'y': 'd', 'z': 'e'}]


In [16]:
h.rewrite('actionGraph', rule, instances[0], rhs_typing=rhs_typing)

CREATE CONSTRAINT ON (n:actionGraph) ASSERT n.id IS UNIQUE
// Match nodes and edges of the instance 
MATCH (lhs_w:actionGraph { id : 'c'}) , (lhs_x:actionGraph { id : 'a'}) , (lhs_y:actionGraph { id : 'd'}) , (lhs_z:actionGraph { id : 'e'})  , (lhs_x)-[lhs_x_lhs_w:edge]->(lhs_w), (lhs_y)-[lhs_y_lhs_x:edge]->(lhs_x), (lhs_y)-[lhs_y_lhs_z:edge]->(lhs_z), (lhs_z)-[lhs_z_lhs_z:edge]->(lhs_z)


// Renaming vars to correspond to the vars of P
WITH lhs_y_lhs_x, lhs_x_lhs_w, lhs_z_lhs_z, lhs_y_lhs_z , lhs_w as p_w, lhs_x as p_x, lhs_y as p_y, lhs_z as p_z 

// Merging nodes '{'w', 'z'}' of the preserved part into 'w_z' 
// accumulate all the attrs of the nodes to be merged
WITH [] as new_props, p_z, lhs_y_lhs_x, p_w, lhs_x_lhs_w, lhs_z_lhs_z, lhs_y_lhs_z, p_y, p_x
WITH new_props + REDUCE(pairs = [], k in keys(p_w) | 
	pairs + REDUCE(inner_pairs = [], v in p_w[k] | 
		inner_pairs + {key: k, value: v})) as new_props, p_z, lhs_y_lhs_x, p_w, lhs_x_lhs_w, lhs_z_lhs_z, lhs_y_lhs_z, p_y, p_x
WITH new

(<regraph.neo4j.hierarchy.Neo4jHierarchy at 0x7fc5fbdc17b8>,
 {'x': 'a', 'y': 'd', 'new_node': '166408', 'w_z': '166404'})

## Removing a graph from the hierarchy

We can also remove a graph from the hierarchy. All its nodes and connections (edges and typing edges) are removed.